In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from env import *
import random
from tqdm import tqdm

In [3]:
env = robot_goal()

In [8]:
def Random_games():
    # Each of this episode is its own game.
    for episode in range(3):
        print("Episode: ", episode)
        reward_list = []
        env.reset()
        simulation_start = 0
        while simulation_start != 1:
            simulation_start = env.sim.startSimulation()
        # this is each frame, up to 500...but we wont make it that far with random.
        for t in tqdm(range(15)):
            action = np.array([8*random.random() - 8*random.random() ,np.pi*random.random() - np.pi*random.random()])
            next_state, reward, done = env.step(action)
            reward_list.append(reward)
            if done:
                break
        simulation_end = 0        
        while simulation_end != 1:
            simulation_end = env.sim.stopSimulation()        
Random_games()

Episode:  0


 60%|██████    | 9/15 [00:07<00:05,  1.16it/s]


True
Episode:  1


 60%|██████    | 9/15 [00:07<00:05,  1.18it/s]


True
Episode:  2


 60%|██████    | 9/15 [00:07<00:05,  1.19it/s]

True
